In [1]:
import networkx as nx
from itertools import groupby
from functools import cache

In [2]:
with open('../data/2024/day21.txt') as f:
    data = f.read()

In [3]:
codes = data.splitlines()

In [4]:
# Number pad adjacency (favor: left, down, right, up)
numpad_adj = """
A0<,A3^,0A>,02^,12>,14^,21<,20v,23>,25^,32<,3Av,36^,41v,45>,47^,
54<,52v,56>,58^,65<,63v,69^,74v,78>,87<,85v,89>,98<,96v
""".replace('\n','').split(',')
G_numpad = nx.DiGraph()
for u,v,k in list(numpad_adj): G_numpad.add_edge(u,v,key=k)

In [5]:
@cache
def shortest_path_numpad(u, v):
    paths = list(nx.all_shortest_paths(G_numpad, u, v))
    ranks = {'<':0,'v':1,'>':2,'^':3}
    results = []

    for path in paths:
        dirs = []
        for a, b in zip(path, path[1:]):
            dirs.append(G_numpad[a][b]['key'])
        results.append(dirs)

    # Favor results that move in straight lines in preferred directional order
    results = sorted(results, key=lambda r: (len(r), len([k for k,_ in groupby(r)]), ranks[r[0]]))

    return results[0]

In [6]:
# Directional pad
dirpad_adj = "A^<,A>v,^A>,^vv,v>>,v<<,v^^,<v>,>A^,>v<".split(',')
G_dirpad = nx.DiGraph()
for u,v,k in list(dirpad_adj): G_dirpad.add_edge(u,v,key=k)

In [7]:
@cache
def shortest_path_dirpad(u, v):
    paths = list(nx.all_shortest_paths(G_dirpad, u, v))
    ranks = {'<':0,'v':1,'^':2,'>':3}
    results = []

    for path in paths:
        dirs = []
        for a, b in zip(path, path[1:]):
            dirs.append(G_dirpad[a][b]['key'])
        results.append(dirs)

    # Favor results that move in straight lines
    if results != [[]]:
        results = sorted(results, key=lambda r: (len(r), len([k for k,_ in groupby(r)]), ranks[r[0]]))

    return results[0]

In [8]:
@cache
def robo_inception(seq: list, depth: int):
    seq_len = 0

    for u, v in zip(seq, seq[1:]):
        uv_path = shortest_path_dirpad(u, v) + ['A']

        # Only return the length of the deepest layer
        if 0 == depth:
            seq_len += len(uv_path)
        else:
            seq_len += robo_inception(tuple(['A'] + uv_path), depth-1)

    return seq_len

In [9]:
def type_code(depth: int):
    count = 0

    for code in codes:
        start_code = list('A' + code)
        code_len = 0

        for u, v in zip(start_code, start_code[1:]):
            step = shortest_path_numpad(u, v) + ['A']
            code_len += robo_inception(tuple(['A'] + step), depth-1)

        count += code_len * int(''.join([d for d in code if d.isdigit()]))

    return count

In [10]:
print("Part 1:", type_code(2))
#print("Part 2:", type_code(25))

Part 1: 164960
